# Part 4: Score the trained model



Microsoft Fabric allows you to operationalize machine learning models with a scalable function called PREDICT, which supports batch scoring in any compute engine. You can generate batch predictions directly from a Microsoft Fabric notebook or from a given model's item page. Learn about [PREDICT](https://aka.ms/fabric-predict).  

To generate batch predictions on our test dataset, you'll use version 1 of the trained churn model. You'll load the test dataset into a spark DataFrame and create an MLFlowTransformer object to generate batch predictions. You can then invoke the PREDICT function using one of following three ways: 

- Using the Transformer API from SynapseML
- Using the Spark SQL API
- Using PySpark user-defined function (UDF)

## Load the test data

Load the test data that you saved in Part 3.

In [1]:
df_test = spark.read.format("delta").load("Tables/df_test")
display(df_test)

StatementMeta(, 077d0ae0-f7dd-4319-8abe-fc0392b29892, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5090668b-18fd-4e66-817c-7b3b1abb4a29)

### PREDICT with the Transformer API

To use the Transformer API from SynapseML, you'll need to first create an MLFlowTransformer object.

### Instantiate MLFlowTransformer object

The MLFlowTransformer object is a wrapper around the MLFlow model that you registered in Part 3. It allows you to generate batch predictions on a given DataFrame. To instantiate the MLFlowTransformer object, you'll need to provide the following parameters:

- The columns from the test DataFrame that you need as input to the model (in this case, you would need all of them).
- A name for the new output column (in this case, predictions).
- The correct model name and model version to generate the predictions (in this case, `lgbm_sm` and version 1).

In [2]:
from synapse.ml.predict import MLFlowTransformer

model = MLFlowTransformer(
    inputCols=list(df_test.columns),
    outputCol='predictions',
    modelName='lgbm_sm',
    modelVersion=1
)

StatementMeta(, 077d0ae0-f7dd-4319-8abe-fc0392b29892, 4, Finished, Available, Finished)

2025/05/02 07:39:21 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - scikit-learn (current: 1.2.2, required: scikit-learn==1.6.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of speci

StatementMeta(, 077d0ae0-f7dd-4319-8abe-fc0392b29892, 9, Finished, Available, Finished)

Now that you have the MLFlowTransformer object, you can use it to generate batch predictions.

In [3]:
import pandas

predictions = model.transform(df_test)
display(predictions)

StatementMeta(, 077d0ae0-f7dd-4319-8abe-fc0392b29892, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 03bb2d38-5e04-44f4-8ca3-cb511f3195a5)

### PREDICT with the Spark SQL API

In [4]:
from pyspark.ml.feature import SQLTransformer 

# Substitute "model_name", "model_version", and "features" below with values for your own model name, model version, and feature columns
model_name = 'lgbm_sm'
model_version = 1
features = df_test.columns

sqlt = SQLTransformer().setStatement( 
    f"SELECT PREDICT('{model_name}/{model_version}', {','.join(features)}) as predictions FROM __THIS__")

# Substitute "X_test" below with your own test dataset
display(sqlt.transform(df_test))

StatementMeta(, 077d0ae0-f7dd-4319-8abe-fc0392b29892, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6ca543ff-a31c-4cad-8d31-2930d5644d6a)

### PREDICT with a user-defined function (UDF)

In [5]:
from pyspark.sql.functions import col, pandas_udf, udf, lit

# Substitute "model" and "features" below with values for your own model name and feature columns
my_udf = model.to_udf()
features = df_test.columns

display(df_test.withColumn("predictions", my_udf(*[col(f) for f in features])))

StatementMeta(, 077d0ae0-f7dd-4319-8abe-fc0392b29892, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e067e56d-288d-4541-8ed5-dd30d1d3cd34)

## Write model prediction results to the lakehouse

Once you have generated batch predictions, write the model prediction results back to the lakehouse.  

In [6]:
# Save predictions to lakehouse to be used for generating a Power BI report
table_name = "customer_churn_test_predictions"
predictions.write.format('delta').mode("overwrite").save(f"Tables/{table_name}")
print(f"Spark DataFrame saved to delta table: {table_name}")


StatementMeta(, 077d0ae0-f7dd-4319-8abe-fc0392b29892, 8, Finished, Available, Finished)

Spark DataFrame saved to delta table: customer_churn_test_predictions
